In [1]:
import sys
sys.path.append("code")

# %load_ext autoreload

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
import pandas as pd
import numpy as np
import seaborn as sb

from pathlib import Path

In [3]:
from spice_tools import init_spice, as_et, as_utc, as_tdb
from simulation_tools import load_all_simulation_results
from simulation_tools import pos_names, acc_names, irr_names, panels_count_names
from plotting import format_plot, save_plot

In [4]:
results_base = Path("../results/2023-01-27T10-00-57-77048")

In [5]:
metadata = load_all_simulation_results(results_base, load_runs=False)
# metadata, runs = load_all_simulation_results(results_base, load_runs=True)

100%|██████████| 104/104 [00:00<00:00, 215.89it/s]


In [6]:
metadata

,id,start_timestamp,save_dir,simulation_start,target_type,use_occultation,use_moon_radiation,number_of_panels_moon,thermal_type,use_instantaneous_reradiation,simulation_duration,step_size,walltime_duration
1,51566,2023-01-27T10:00:57.047861,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Paneled,False,True,2000,Delayed,True,33900,10,635.76902
2,9f421,2023-01-27T10:00:57.047879,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 SEP 26 06:00:00,Paneled,False,True,2000,AngleBased,True,33900,10,637.876791
3,e85a2,2023-01-27T10:00:57.047902,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Paneled,False,True,20000,AngleBased,True,33900,10,1902.433609
4,fb222,2023-01-27T10:00:57.047917,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Paneled,False,False,0,,False,33900,10,500.656141
5,69f65,2023-01-27T10:00:57.047929,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 SEP 26 06:00:00,Cannonball,False,True,5000,AngleBased,True,33900,10,752.347974
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3fde7,2023-01-27T10:00:57.048636,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Cannonball,True,True,20000,Delayed,False,33900,10,2156.773656
101,1f1bd,2023-01-27T10:00:57.048643,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Cannonball,False,True,5000,AngleBased,True,33900,10,1066.784294
102,ac7ca,2023-01-27T10:00:57.048649,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Paneled,True,True,2000,AngleBased,False,33900,10,907.099581
103,04c45,2023-01-27T10:00:57.048656,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 SEP 26 06:00:00,Cannonball,False,True,5000,Delayed,False,33900,10,1053.777965


In [7]:
metadata[
    (metadata["simulation_start"] == "2010 JUN 26 06:00:00") &
    (metadata["target_type"] == "Paneled") &
    (metadata["use_moon_radiation"]) &
    (metadata["use_occultation"]) &
    (metadata["number_of_panels_moon"] == 20000) &
    (metadata["thermal_type"] == "AngleBased") &
    (metadata["use_instantaneous_reradiation"] == False)
]

,id,start_timestamp,save_dir,simulation_start,target_type,use_occultation,use_moon_radiation,number_of_panels_moon,thermal_type,use_instantaneous_reradiation,simulation_duration,step_size,walltime_duration
95,5c010,2023-01-27T10:00:57.048598,/home2/dominik/dev/lropy/results/2023-01-27T10...,2010 JUN 26 06:00:00,Paneled,True,True,20000,AngleBased,False,33900,10,2579.018326
